<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500 height=450/></p>

<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Продвинутый поток. Весна 2021</b></h3>

Это домашнее задание будет посвящено полноценному решению задачи машинного обучения.

Есть две части этого домашнего задания: 
* Сделать полноценный отчет о вашей работе: как вы обработали данные, какие модели попробовали и какие результаты получились (максимум 10 баллов). За каждую выполненную часть будет начислено определенное количество баллов.
* Лучшее решение отправить в соревнование на [kaggle](https://www.kaggle.com/c/advanced-dls-spring-2021/) (максимум 5 баллов). За прохождение определенного порогов будут начисляться баллы.


**Обе части будут проверяться в формате peer-review. Т.е. вашу посылку на степик будут проверять несколько других студентов и аггрегация их оценок будет выставлена. В то же время вам тоже нужно будет проверить несколько других учеников.**

**Пожалуйста, делайте свою работу чистой и понятной, чтобы облегчить проверку. Если у вас будут проблемы с решением или хочется совета, то пишите в наш чат в телеграме или в лс @runfme. Если вы захотите проаппелировать оценку, то пипшите в лс @runfme.**

**Во всех пунктах указания это минимальный набор вещей, которые стоит сделать. Если вы можете сделать какой-то шаг лучше или добавить что-то свое - дерзайте!**

# Как проверять?

Ставьте полный балл, если выполнены все рекомендации или сделано что-то более интересное и сложное. За каждый отсустствующий пункт из рекомендация снижайте 1 балл.

# Метрика

Перед решением любой задачи важно понимать, как будет оцениваться ваше решение. В данном случае мы используем стандартную для задачи классификации метрику ROC-AUC. Ее можно вычислить используя только предсказанные вероятности и истинные классы без конкретного порога классификации + она работает даже если классы в данных сильно несбалансированны (примеров одного класса в десятки раз больше примеров другого). Именно поэтому она очень удобна для соревнований.

Посчитать ее легко:


In [ ]:
from sklearn.metrics import roc_auc_score

y_true = [
    0,
    1,
    1,
    0,
    1
]

y_predictions = [
    0.1,
    0.9,
    0.4,
    0.6,
    0.61
]

roc_auc_score(y_true, y_predictions)

# Первая часть. Исследование

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Загрузка данных (2 балла)

1) Посмотрите на случайные строчки. 

2) Посмотрите, есть ли в датасете незаполненные значения (nan'ы) с помощью data.isna() или data.info() и, если нужно, замените их на что-то. Будет хорошо, если вы построите табличку с количеством nan в каждой колонке.

In [ ]:
train_path = r'../input/advanced-dls-spring-2021/train.csv'
# data = pd.read_csv('./train.csv')
data = pd.read_csv(train_path)

In [ ]:
# Для вашего удобства списки с именами разных колонок

# Числовые признаки
num_cols = [
    'ClientPeriod',
    'MonthlySpending',
    'TotalSpent'
]

# Категориальные признаки
cat_cols = [
    'Sex',
    'IsSeniorCitizen',
    'HasPartner',
    'HasChild',
    'HasPhoneService',
    'HasMultiplePhoneNumbers',
    'HasInternetService',
    'HasOnlineSecurityService',
    'HasOnlineBackup',
    'HasDeviceProtection',
    'HasTechSupportAccess',
    'HasOnlineTV',
    'HasMovieSubscription',
    'HasContractPhone',
    'IsBillingPaperless',
    'PaymentMethod'
]

feature_cols = num_cols + cat_cols
target_col = 'Churn'

Посмотрим на размер данных и на то, как они выглядят:

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

Выведем информацию о датасете:


In [ ]:
data.info()

Пропущенных значений вроде бы нет, однако есть смысл проверить числовые колонки на 0, а остальные - на пустые строки. Кроме того, колонки TotalSpent и IsSeniorCitizen имеют неожиданный формат. Разберемся с этим позже, для начала посмотрим на значения, которые могут быть пропущены:


In [ ]:
data[data['ClientPeriod'] == 0].shape

In [ ]:
data[data.TotalSpent == ' '].shape

Мы видим одинаковое число предположительно пропущенных значений в столбцах TotalSpent и ClientPeriod. Интересно, это одни и те же данные или нет?


In [ ]:
index_missing_TotalSpent = data[data.TotalSpent == ' '].index
index_missing_TotalSpent

In [ ]:
index_missing_TotalSpent == data[data['ClientPeriod']==0].index # да, это одни и те же строки

In [ ]:
data.iloc[index_missing_TotalSpent]

Итак, у нас в обучающем наборе есть данные с нулевым периодом, у которых также не заполнена колонка TotalSpent и их совсем немного (доля процента). Чтобы понять, что с ними делать дальше (обрабатывать или удалять), взглянем, а есть ли такие объекты в тестовых данных:


In [ ]:
test_path = '../input/advanced-dls-spring-2021/test.csv'
# test_data = pd.read_csv('./test.csv')
test_data = pd.read_csv(test_path)
test_data.shape

In [ ]:
test_data[test_data.TotalSpent == ' '].shape

In [ ]:
test_data[test_data['ClientPeriod']==0].shape

In [ ]:
test_data.info()

В тестовых данных у нас тоже есть подобные объекты, а кроме того, такая же путаница с колонками TotalSpent и IzSeniorCitizen. Значит, удалять ничего нельзя, и нужно предусмотреть преобразования "испорченных"  колонок для обоих наборов данных.  Но прежде посмотрим на значения остальных колонок, все ли с ними в порядке:

In [ ]:
for col in cat_cols:
  print(f"Column '{col}':")
  print(data[col].value_counts(), end='\n')
  print('----------------')


Похоже, пропусков больше нигде нет. Создадим свой простенький класc-трансформер для преобразования двух выявленных некорректных признаков:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


In [ ]:
class ColumnTypeTransformer(BaseEstimator, TransformerMixin):
    '''Преобразовывает колонку TotalSpent в числовой тип,
    а колонку IsSeniorCitizen - в строковый'''

    def __init__(self):  # без *args и **kargs
        self.to_str_column = 'IsSeniorCitizen'
        self.to_float_column = 'TotalSpent'

    def fit(self, X, y=None):
        return self  # больше ничего делать не надо

    def transform(self, X):
        # заменяем пробелы на нули
        X.loc[X[self.to_float_column] == ' ', self.to_float_column] = '0'
        # преобрзовываем числовой признак в действительное число
        X[self.to_float_column] = X[self.to_float_column].astype(float)
        # а категориальный признак - в строку
        X[self.to_str_column] = X[self.to_str_column].astype(str)

        return X

Поскольку данные с корретными типами колонок нам понадобятся ДО дальнейшей обработки (для построения графиков), сразу же создадим корректные обучающий и тестовый наборы, которые в дальнейшем и будем использовать:

In [ ]:
type_transformer = ColumnTypeTransformer()
X_train = type_transformer.transform(data)
X_train.info() # сразу же проверяем корректность

In [ ]:
# теперь разделим на X и y
y_train = X_train[target_col]
X_train = X_train.drop([target_col], axis=1)
X_train.shape

In [ ]:
# преобразуем тестовые данные
X_test = type_transformer.transform(test_data)
X_test.info() # сразу же проверяем корректность

Наши данные исправлены и готовы к использованию.

## Анализ данных (3 балла)

1) Для численных призанков постройте гистограмму (*plt.hist(...)*) или boxplot (*plt.boxplot(...)*). Для категориальных посчитайте количество каждого значения для каждого признака. Для каждой колонки надо сделать *data.value_counts()* и построить bar диаграммы *plt.bar(...)* или круговые диаграммы *plt.pie(...)* (хорошо, елси вы сможете это сделать на одном гарфике с помощью *plt.subplots(...)*). 

2) Посмотрите на распределение целевой переменной и скажите, являются ли классы несбалансированными.

3) (Если будет желание) Поиграйте с разными библиотеками для визуализации - *sns*, *pandas_visual_analysis*, etc.

Второй пункт очень важен, потому что существуют задачи классификации с несбалансированными классами. Например, это может значить, что в датасете намного больше примеров 0 класса. В таких случаях нужно 1) не использовать accuracy как метрику 2) использовать методы борьбы с imbalanced dataset (обычно если датасет сильно несбалансирован, т.е. класса 1 в 20 раз меньше класса 0).

In [ ]:
# смотрим распределения числовых колонок, их у нас 3
X_train.hist(column=num_cols, figsize=(20, 8), bins = 20)
# X_train.boxplot(column=num_cols, figsize=(14, 10))
None


In [ ]:
# смотрим гистограммы категориальных признаков, их у нас 19-3 = 16
fig, ax = plt.subplots(4, 4, figsize=(18,18))
ax = ax.ravel()

for i, col in enumerate(cat_cols):
  ax[i].set_title(col)
  col_values = X_train[col].value_counts()
  ax[i].bar(col_values.index, col_values.values)
  ax[i].set_xticklabels(col_values.index, rotation = 30)
  

plt.subplots_adjust(hspace = 0.5)
fig.show()

Видим, что масштаб данных различен и для применения линейных моделей их стоит отмасштабировать. 

In [ ]:
# смотрим распределение целевой переменной
fig = plt.figure(figsize = (3, 5))
ax0 = fig.add_subplot(111)
target_values = y_train.value_counts()
ax0.bar(target_values.index, target_values.values)


In [ ]:
target_values

Видим некоторый дисбаланс классов целевой переменной, хотя и не очень большой. Но все же метрики вроде accuracy использовать не стоит... И, соответственно, для классификации стоит использовать только "вероятностные" модели. 

(Дополнительно) Если вы нашли какие-то ошибки в данных или выбросы, то можете их убрать. Тут можно поэксперементировать с обработкой данных как угодно, но не за баллы.

In [ ]:
# YOUR CODE

## Применение линейных моделей (3 балла)

1) Обработайте данные для того, чтобы к ним можно было применить LogisticRegression. Т.е. отнормируйте числовые признаки, а категориальные закодируйте с помощью one-hot-encoding'а. 

2) С помощью кроссвалидации или разделения на train/valid выборку протестируйте разные значения гиперпараметра C и выберите лучший (можно тестировать С=100, 10, 1, 0.1, 0.01, 0.001) по метрике ROC-AUC. 

Если вы разделяете на train/valid, то используйте LogisticRegressionCV. Он сам при вызове .fit() подберет параметр С. (не забудьте передать scroing='roc_auc', чтобы при кроссвалидации сравнивались значения этой метрики, и refit=True, чтобы при потом модель обучилась на всем датасете с лучшим параметром C). 


(более сложный вариант) Если вы будете использовать кроссвалидацию, то преобразования данных и LogisticRegression нужно соединить в один Pipeline с помощью make_pipeline, как это делалось во втором семинаре. Потом pipeline надо передать в GridSearchCV. Для one-hot-encoding'a можно испльзовать комбинацию LabelEncoder + OneHotEncoder (сначала превращаем строчки в числа, а потом числа првращаем в one-hot вектора.)

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline

# еще кое-что импортируем
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


Создадим конвейер для предобработки данных и подбора параметров. Для обработки разных столбцов будем использовать ColumnTransformer(), а для подбора параметров - GridSearchCV(). Соответственно, вручную разделять датасет на train/test не нужно, поскольку GridSearchCV использует кроссвалидацию и позаботиться о разбивке датасета сам. Число фолдов оставим по умолчанию:

In [ ]:

prepare_pipeline = ColumnTransformer([
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(), cat_cols)
    ])

full_pipeline = Pipeline([
        ("preparation", prepare_pipeline),
        ("linear", LogisticRegression(solver = 'liblinear', random_state=42))
    ])



In [ ]:
# посмотрим на имена параметров, чтобы задать сетку для поиска

full_pipeline.get_params()


Попробуем подобрать для логистическоей регрессии вариант регуляризации (лассо, ридж) и С - коээффициент регуляризации:

In [ ]:

C_values_lst = [120, 100, 10, 1, 0.1, 0.01, 0.001]

param_grid = [
        {'linear__penalty':['l1', 'l2'],
         'linear__C':C_values_lst
         }
    ]


In [ ]:
grid_search = GridSearchCV(full_pipeline, 
                          param_grid, 
                          scoring='roc_auc', 
                          verbose=4)



In [ ]:
grid_search.fit(X_train, y_train)

Для LogisticRegression() можно еще попробовать ElasticNet и поискать соответствующие коэффициенты. Также можно попробовать удалить  один из бинарных признаков в OneHotEncoder() (чтобы избежать их линейной зависимости - это может улучшить линейную модель), и поиграться с вариантами солверов (ибо они работают по разному). 

Это было проделано. Но поскольку эластик нет (и l1 на других солверах) работают хуже, удаление одного из бинарных признаков в OneHotEncoder() также не улучшает скор, то не буду загромождать отчет, ограничившись кратким перечислением результатов на обучающей выборке:

 - для эластикнет с солвером saga и параметрами
  {'linear__C': 100,
  'linear__l1_ratio': 0.5,
  'linear__penalty': 'elasticnet',
  'preparation__cat__drop': None}
  результат: 0,845132

 - для солвера saga с параметрами 
  {'linear__C': 120, 'linear__penalty': 'l2'}
  результат: 0,845129

 - для солвера 'liblinear' с параметрами
  {'linear__C': 100, 'linear__penalty': 'l1'}
  результат:  0.8451325 - мой выбор (просто и быстро) 


Итак, лучшая полученная модель с параметрами:


In [ ]:
# лучшие параметры
grid_search.best_params_

In [ ]:
# лучший скор
grid_search.best_score_


In [ ]:
# лучшая модель
grid_search.best_estimator_

**ВЫВОД:**


**Лучший roc_auc_score для логистической регрессии на валидационной выборке равен  0.84513 и получен для следующих параметров: C=100 (в колабе, где делался блокнот), penalty='l1' (при solver='liblinear').**

**При отправке на Kaggle score on PublicLiderBoard = 0.84583, т. е. скор практически аналогичен скору на обучающей выборке, что говорит о том, что параметры модели подобраны удачно и она не переобучилась... Таким образом,
roc auc > 0.84 может быть достигнут уже на логистической регрессии, при этом поиск параметров занимает время, измеряемое секундами.**



In [ ]:
# код для формирования файла submission.csv для отправки на Kaggle
# использовался С=100!!! (а не 120, как насчитал блокнот на Kaggle)

# best_pipeline = grid_search.best_estimator_
# predictions = best_pipeline.predict_proba(X_test)

# submission = pd.read_csv('./submission.csv', index_col=0)
# submission['Churn'] = predictions[:, 1]
# submission.to_csv('./my_submission2.csv')

## Применение градиентного бустинга (2 балла)

Если вы хотите получить баллы за точный ответ, то стоит попробовать градиентный бустинг. Часто градиентный бустинг с дефолтными параметрами даст вам 80% результата за 0% усилий.

Мы будем использовать catboost, поэтому нам не надо кодировать категориальные признаки. catboost сделает это сам (в .fit() надо передать cat_features=cat_cols). А численные признаки нормировать для моделей, основанных на деревьях не нужно.

1) Разделите выборку на train/valid. Протестируйте catboost cо стандартными параметрами.

2) Протестируйте разные занчения параметроа количества деревьев и learning_rate'а и выберите лучшую по метрике ROC-AUC комбинацию. 

(Дополнительно) Есть некоторые сложности с тем, чтобы использовать CatBoostClassifier вместе с GridSearchCV, поэтому мы не просим использовать кроссвалидацию. Но можете попробовать)

In [ ]:
# при работе в Colab, или при неустановленном CatBoost раскомментировать
!pip install catboost

In [ ]:
# встроенная в catboost визуализация в Colab не работает даже при установке требуемых пакетов :(
# !pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension

In [ ]:
from catboost import CatBoostClassifier
from catboost import Pool, cv 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
# получим список имен колонок для создания датасета CatBoost
feature_names = list(X_train.columns)


Попробуем сначала с параметрами по умолчанию:

In [ ]:
# без грид серча сначала попробуем,
# для этого сами разделим датасет и посмотрим результаты обучения

X_train_cb, X_test_cb, y_train_cb, y_test_cb = train_test_split(X_train.values, y_train.values, 
                                                       train_size=0.8, 
                                                       random_state=42)

In [ ]:
catboost_train_splitted = Pool(data=X_train_cb,
                  label=y_train_cb,
                  cat_features=cat_cols,
                  feature_names=feature_names
                  )

In [ ]:
# cv Не будем использовать...
# просто образец кода для cv, встроенной в CatBoost

# params = {
# 'loss_function' : 'Logloss',
# 'eval_metric' : 'AUC',
# 'verbose' : 200,
# 'random_seed' : 42,
# 'custom_metric' : 'AUC:hints=skip_train~false'
# }

# cbc = CatBoostClassifier(**params)
# cbc.fit(x_tr, y_tr,
#         eval_set = (x_te, y_te),
#         use_best_model = True,
#         plot = True
#         );

# predictions = cbc.predict(x_te)

In [ ]:
boosting_model = CatBoostClassifier(loss_function='Logloss', 
                                    random_state=42,
                                    verbose=False,
                                    eval_metric='AUC',
                                    custom_metric='AUC:hints=skip_train~false'                                
                                    )

boosting_model.fit(catboost_train_splitted)

y_train_splitted_predicted = boosting_model.predict_proba(X_train_cb)[:, 1]
y_test_splitted_predicted = boosting_model.predict_proba(X_test_cb)[:, 1]

In [ ]:
train_auc = roc_auc_score(y_train_cb, y_train_splitted_predicted)
test_auc = roc_auc_score(y_test_cb, y_test_splitted_predicted)

plt.figure(figsize=(10,7))
plt.plot(*roc_curve(y_train_cb, y_train_splitted_predicted)[:2], label='train AUC={:.4f}'.format(train_auc))
plt.plot(*roc_curve(y_test_cb, y_test_splitted_predicted)[:2], label='test AUC={:.4f}'.format(test_auc))
legend_box = plt.legend(fontsize='large', framealpha=1).get_frame()
legend_box.set_facecolor("white")
legend_box.set_edgecolor("black")
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100))
plt.show()
# roc auc на тестовом наборе 0.8214, а на трейне - 0.8981... серьезно переобучились...

Результаты с параметрами по умолчанию слабенькие, плюс видим явное переобучение. Попробуем теперь встроенный поиск параметров на всем обучающем датасете (поскольку обучение будет с кроссвалидацией). Будем искать рекомендованные в задании параметры (iterations и learning_rate), плюс попробуем построить дерево так, чтобы можно было его ограничить (например, задать минимальное количество экземпляров в листе) - для этого подберем соответствующую политику выращивания деревьев (grow_policy):



In [ ]:
catboost_train_dataset = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_cols,
                  feature_names=feature_names
                  )

In [ ]:
searching_model = CatBoostClassifier(loss_function='Logloss', 
                                    random_state=42,
                                    eval_metric='AUC:hints=skip_train~false'
                                    )

grid = {"iterations": [100, 200, 300],
        "learning_rate":[0.03, 0.1, 0.15, 0.2, 0.3],
        "grow_policy":['Depthwise'],
        "min_data_in_leaf":[20, 30, 50, 100, 200]}

grid_search_result = searching_model.grid_search(grid, X=catboost_train_dataset, 
                                                 verbose=False, plot=True)



In [ ]:
# лучшие полученные параметры
grid_search_result['params']

In [ ]:
# для сравнения и самопроверки
# смотрим скор на трейне
y_train_predicted = searching_model.predict_proba(X_train)[:, 1]
train_searching_auc = roc_auc_score(y_train, y_train_predicted)
train_searching_auc

# скор на тестовой 0.86863
# скор на Каггл = 0.85501
# близко
# кажется вряд ли удастся добиться результата больше 0.86,
# ибо переобучения почти нет...
# так что, наверное, вести дальнейшие поиски не имеет особого смысла


Мы видим, что скор на трейне с лучшими найденными параметрами ниже, чем был при параметрах по умолчанию - это позволяет предположить, что, скорее всего, переобучение удалось уменьшить. При отправке на Kaggle предположение подтверждается - скор на тестовой выборке 0.85501.

**Выводы: лучшие результаты с использованием модели CatBoost составляют 0.85501 и достигнуты при ограничении древовидной модели. При этом качество на обучающей и тестовой выборке очень похожее, что позволяет предположить, что переобучения почти нет, и серьезного улучшения качества добиться вряд ли удастся. Но при большом желании можно попробовать еще и регуляризацию (поскольку она показывала хорошие результаты на логистической регрессии) - может быть удастся получить прибавку в сотые или десятые доли процента.**

*Имя на Kaggle: Анна_Кисман, скор на Kaggle - 0.85501*

# Предсказания

In [ ]:
# y_test_predicted = searching_model.predict_proba(X_test)[:, 1]

# submission = pd.read_csv('./submission.csv', index_col=0)
# submission['Churn'] = y_test_predicted
# submission.to_csv('./my_submission6.csv')

# Kaggle (5 баллов)

Как выставить баллы:

1) 1 >= roc auc > 0.84 это 5 баллов

2) 0.84 >= roc auc > 0.7 это 3 балла

3) 0.7 >= roc auc > 0.6 это 1 балл

4) 0.6 >= roc auc это 0 баллов


Для выполнения задания необходимо выполнить следующие шаги.
* Зарегистрироваться на платформе [kaggle.com](kaggle.com). Процесс выставления оценок будет проходить при подведении итогового рейтинга. Пожалуйста, укажите во вкладке Team -> Team name свои имя и фамилию в формате Имя_Фамилия (важно, чтобы имя и фамилия совпадали с данными на Stepik).
* Обучить модель, получить файл с ответами в формате .csv и сдать его в конкурс. Пробуйте и экспериментируйте. Обратите внимание, что вы можете выполнять до 20 попыток сдачи на kaggle в день.
* После окончания соревнования отправить в итоговый ноутбук с решением на степик. 
* После дедлайна проверьте посылки других участников по критериям. Для этого надо зайти на степик, скачать их ноутбук и проверить скор в соревновании.